# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [ ]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
def build_classifiers():
    
    neighbors = KNeighborsClassifier()
    neighbors.fit(data_set, labels)

    linear_regression = LinearRegression()
    linear_regression.fit(data_set, labels)

    qda = QuadraticDiscriminantAnalysis()
    qda.fit(data_set, labels)
    
    suppvec=SVC()
    suppvec.fit(data_set,labels)
    
    #dt=DecisionTreeClassifier()
    #dt.fit(data_set,labels)
    
    gnb=GaussianNB()
    gnb.fit(data_set,labels)
    
    return neighbors, linear_regression, qda, suppvec, gnb


In [ ]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [ ]:
result=0
wynik=[0,0,0]
dictionary=("neighbors","linear_reg","qda","svm","gnb")
classifiers = build_classifiers()
#print(type(classifiers))
for i in range(5):
    for j in range(i+1,5):
        for k in range(j+1,5):
            #wybrane=(i,j,k)
            wybrane=[classifiers[m] for m in (i, j, k)]
            #print(type(wybrane))
            predicted = build_stacked_classifier(wybrane)
            
            accuracy = accuracy_score(test_labels, predicted)
            #print(accuracy)
            if accuracy>result:
                result=accuracy
                wynik=[i,j,k]
            
print(result)
#print(wynik)
print([dictionary[i] for i in wynik])

0.95
[0, 2, 4]
['neighbors', 'qda', 'gnb']


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [28]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size
#print(weights)

def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [29]:
def set_new_weights(model):
    predicted=np.array(model.predict(test_set))
    #print(predicted)
    #mozemy tez skorzystac z calculate_accuracy_vector(model.predict(test_set), test_labels)
    licznik=1+np.array(1*(predicted!=test_labels))
    #print(licznik)
    mianownik=np.sum(licznik)
    weights=licznik/mianownik
    return weights

Train the classifier with the code below:

In [30]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
model = train_model(classifier, weights)
#print(set_new_weights(model))
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    #print(weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00066578 0.00133156 0.00133156 0.00066578 0.00133156 0.00066578
 0.00066578 0.00133156 0.00066578 0.00066578 0.00133156 0.00066578
 0.00066578 0.00066578 0.00066578 0.00066578 0.00066578 0.00133156
 0.00066578 0.00133156 0.00133156 0.00066578 0.00133156 0.00133156
 0.00066578 0.00066578 0.00133156 0.00133156 0.00066578 0.00133156
 0.00066578 0.00066578 0.00066578 0.00066578 0.00133156 0.00066578
 0.00133156 0.00066578 0.00066578 0.00133156 0.00066578 0.00066578
 0.00133156 0.00066578 0.00066578 0.00066578 0.00066578 0.00133156
 0.00133156 0.00133156 0.00133156 0.00066578 0.00066578 0.00133156
 0.00133156 0.00133156 0.00066578 0.00133156 0.00133156 0.00133156
 0.00066578 0.00066578 0.00066578 0.00133156 0.00066578 0.00133156
 0.00066578 0.00066578 0.00133156 0.00066578 0.00133156 0.00066578
 0.00133156 0.00133156 0.00066578 0.00066578 0.00133156 0.00133156
 0.00066578 0.00133156 0.00133156 0.00133156 0.00133156 0.00133156
 0.00133156 0.00133156 0.00133156 0.00133156 0.00066578 0.0006

Set the validation data set:

In [31]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [32]:
#analogicznie jak w przypadku baggingu, ponieważ odpowiednie wagi są już uwzględnione w tablicy "classifiers" i przewidywania to uwzględniają
def get_prediction(x):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(x))
    output = np.array(output)
    predicted = []
    for i in range(len(x)):
        classified = output[:, i]
        counts = np.bincount(classified)
        predicted.append(np.argmax(counts))
    return predicted


Test it:

In [33]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
